# Fully connected feedforward network implementing a loss mask

In [ ]:
import torchmetrics.classification
from torch import cuda
assert cuda.is_available()
assert cuda.device_count() > 0

In [ ]:
print(cuda.get_device_name(cuda.current_device()))


NVIDIA GeForce RTX 3060 Ti


In [ ]:
import torch
import torch.nn as nn

In [ ]:
DEVICE = torch.device("cuda")
SEED = 76436278

torch.manual_seed(SEED)

### Load the Dataset

In [ ]:
from src.maldi2resistance.data.driams import Driams

driams = Driams(
    root_dir="/home/jan/Uni/master/data/Driams",
)
umg = Driams(
    root_dir="/home/jan/Uni/master/data/Driams",
    bin_size=1,
    sites=["UMG"],
    years=[2020,2021],
    antibiotics=driams.selected_antibiotics,
)

driams.loading_type = "memory"
umg.loading_type = "memory"

driams

/home/jan/Uni/master/src/maldi2resistance/data/driams.py:86: UserWarning: Metafile for Site (DRIAMS-B) and Year (2015) combination not found!
  warnings.warn(
/home/jan/Uni/master/src/maldi2resistance/data/driams.py:86: UserWarning: Metafile for Site (DRIAMS-B) and Year (2016) combination not found!
  warnings.warn(
/home/jan/Uni/master/src/maldi2resistance/data/driams.py:86: UserWarning: Metafile for Site (DRIAMS-B) and Year (2017) combination not found!
  warnings.warn(
/home/jan/Uni/master/src/maldi2resistance/data/driams.py:86: UserWarning: Metafile for Site (DRIAMS-C) and Year (2015) combination not found!
  warnings.warn(
/home/jan/Uni/master/src/maldi2resistance/data/driams.py:86: UserWarning: Metafile for Site (DRIAMS-C) and Year (2016) combination not found!
  warnings.warn(
/home/jan/Uni/master/src/maldi2resistance/data/driams.py:86: UserWarning: Metafile for Site (DRIAMS-C) and Year (2017) combination not found!
  warnings.warn(
/home/jan/Uni/master/src/maldi2resistance/data

Loading Spectra into Memory:   0%|          | 0/55780 [00:00<?, ?it/s]

Loading Spectra into Memory:   0%|          | 0/73745 [00:00<?, ?it/s]

Antibiotic:,Amikacin,Amoxicillin-Clavulanic acid,Ampicillin,Ampicillin-Amoxicillin,Aztreonam,Benzylpenicillin,Cefazolin,Cefepime,Cefpodoxime,Ceftazidime,Ceftriaxone,Cefuroxime,Ciprofloxacin,Clarithromycin,Clindamycin,Colistin,Cotrimoxazole,Ertapenem,Erythromycin,Fosfomycin,Fosfomycin-Trometamol,Fusidic acid,Gentamicin,Imipenem,Levofloxacin,Meropenem,Mupirocin,Nitrofurantoin,Norfloxacin,Oxacillin,Penicillin,Piperacillin-Tazobactam,Polymyxin B,Rifampicin,Teicoplanin,Tetracycline,Tobramycin,Vancomycin
Number resistant:,1068,13366,8578,21966,628,618,4223,7383,2338,3470,8659,5855,9338,310,4381,2874,7405,427,5468,2303,1326,3620,3481,7560,4217,5194,570,1271,1205,5537,12431,7616,486,580,244,3534,1707,227
Number susceptible:,20941,24992,4194,4905,456,457,5813,31567,4382,24566,28464,8368,36822,1262,9841,15784,24590,21740,9044,10184,4803,8498,22662,31717,17989,27228,4656,3603,7031,7740,4286,31308,2305,14964,8486,10376,16809,20540
Number data points:,22009,38358,12772,26871,1084,1075,10036,38950,6720,28036,37123,14223,46160,1572,14222,18658,31995,22167,14512,12487,6129,12118,26143,39277,22206,32422,5226,4874,8236,13277,16717,38924,2791,15544,8730,13910,18516,20767


In [ ]:
len(driams.label_stats.columns)

38

In [ ]:
from maldi2resistance.model.MLP import AeBasedMLP
import copy

model = AeBasedMLP(input_dim=18000, output_dim=len(driams.selected_antibiotics), hidden_dim=4096, latent_dim=2048)
model = model.to(DEVICE)

model_state = copy.deepcopy(model.state_dict()) 

In [ ]:
from torchinfo import summary

print(summary(model))

Layer (type:depth-idx)                   Param #
AeBasedMLP                               --
├─Encoder: 1-1                           --
│    └─Linear: 2-1                       73,732,096
│    └─Linear: 2-2                       16,781,312
│    └─Linear: 2-3                       8,390,656
│    └─LeakyReLU: 2-4                    --
├─Decoder: 1-2                           --
│    └─Linear: 2-5                       8,392,704
│    └─Linear: 2-6                       16,781,312
│    └─Linear: 2-7                       155,686
│    └─LeakyReLU: 2-8                    --
Total params: 124,233,766
Trainable params: 124,233,766
Non-trainable params: 0


In [ ]:
from torch.utils.data import DataLoader

In [ ]:
from maldi2resistance.metric.PrecisionRecall import MultiLabelPRNan
from pathlib import Path
from maldi2resistance.metric.ROC import MultiLabelRocNan
from maldi2resistance.loss.maskedLoss import MaskedBCE
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR

print("Start training ...")
model.train()

batch_size = 128
fig_path = Path("./kfold/figures")
fig_path.mkdir(parents=True, exist_ok=True)

loss_per_batch = []

class_weights_negative = torch.tensor((1 - ((driams.label_stats.loc["negative"] +umg.label_stats.loc["negative"]) / (driams.label_stats.loc["n_sum"] +umg.label_stats.loc["n_sum"]))).values, device=DEVICE)
class_weights_positive = torch.tensor((1 - ((driams.label_stats.loc["positive"] + umg.label_stats.loc["positive"]) / (driams.label_stats.loc["n_sum"]+umg.label_stats.loc["n_sum"]))).values, device=DEVICE)

criterion = MaskedBCE()

gen = torch.Generator()

for fold, (train_data, test_data) in enumerate(driams.getK_fold(n_splits=5, shuffle=True, random_state= SEED)):
    train_data_umg = torch.utils.data.ConcatDataset([train_data, umg])
    
    train_loader = DataLoader(train_data_umg, batch_size=batch_size, shuffle=True, drop_last=True, generator= gen.manual_seed(SEED))
    test_loader = DataLoader(test_data, batch_size=len(test_data), shuffle=True, drop_last=True, generator= gen.manual_seed(SEED))
    
    model.load_state_dict(model_state)
    model.train()
    
    optimizer = Adam(model.parameters(), lr=1e-3, amsgrad = True)
    scheduler = StepLR(optimizer, step_size=10, gamma=0.5)

    for epoch in tqdm(range(30)):
        overall_loss = 0
        
        for batch_idx, (x, y) in enumerate(train_loader):
    
            x = x.to(DEVICE)
            y = y.to(DEVICE)
            
            optimizer.zero_grad()
    
            output = model(x)
        
            loss = criterion(output, y)
            current_loss_value = loss.item()
            loss_per_batch.append(current_loss_value)
            
            overall_loss += current_loss_value
            
            loss.backward()
            optimizer.step()
    
        scheduler.step()
        with tqdm.external_write_mode():
            print(f"\tAverage Loss: {overall_loss / (batch_idx*batch_size):.6f} \tLearning rate: {scheduler.get_last_lr()[0]:.6f}")
    
    
    print(f"Finished Fold {fold}")
    
    model.eval()
    
    DEVICE = torch.device("cpu")
    test_features, test_labels = next(iter(test_loader))
    test_features = test_features.to(DEVICE)
    test_labels = test_labels.to(DEVICE)
    model = model.to(DEVICE)
    
    ml_roc = MultiLabelRocNan()
    output = model(test_features)
    
    
    ml_roc.compute(output,test_labels,driams.selected_antibiotics, create_csv=f"./kfold/csv/fold-{fold}_ROC.csv")
    fig_, ax_ = ml_roc()
    
    plt.savefig(fig_path / f"fold-{fold}_ROC.png", transparent=True, format= "png", bbox_inches = "tight")
    plt.close()
    
    ml_pr = MultiLabelPRNan()
    ml_pr.compute(output,test_labels,driams.selected_antibiotics, create_csv=f"./kfold/csv/fold-{fold}_PrecisionRecall.csv")
    
    fig_, ax_ = ml_pr()
    
    plt.savefig(fig_path / f"fold-{fold}_PrecisionRecall.png", transparent=True, format= "png", bbox_inches = "tight")
    plt.close()
    
    DEVICE = torch.device("cuda")
    model = model.to(DEVICE)

Start training ...


  0%|          | 0/30 [00:00<?, ?it/s]

	Average Loss: 0.000749 	Learning rate: 0.001000
	Average Loss: 0.000604 	Learning rate: 0.001000
	Average Loss: 0.000560 	Learning rate: 0.001000
	Average Loss: 0.000535 	Learning rate: 0.001000
	Average Loss: 0.000516 	Learning rate: 0.001000
	Average Loss: 0.000500 	Learning rate: 0.001000
	Average Loss: 0.000484 	Learning rate: 0.001000
	Average Loss: 0.000471 	Learning rate: 0.001000
	Average Loss: 0.000458 	Learning rate: 0.001000
	Average Loss: 0.000447 	Learning rate: 0.000500
	Average Loss: 0.000402 	Learning rate: 0.000500
	Average Loss: 0.000387 	Learning rate: 0.000500
	Average Loss: 0.000376 	Learning rate: 0.000500
	Average Loss: 0.000365 	Learning rate: 0.000500
	Average Loss: 0.000355 	Learning rate: 0.000500
	Average Loss: 0.000342 	Learning rate: 0.000500
	Average Loss: 0.000332 	Learning rate: 0.000500
	Average Loss: 0.000319 	Learning rate: 0.000500
	Average Loss: 0.000309 	Learning rate: 0.000500
	Average Loss: 0.000300 	Learning rate: 0.000250
	Average Loss: 0.000

  0%|          | 0/30 [00:00<?, ?it/s]

	Average Loss: 0.000749 	Learning rate: 0.001000
	Average Loss: 0.000605 	Learning rate: 0.001000
	Average Loss: 0.000559 	Learning rate: 0.001000
	Average Loss: 0.000535 	Learning rate: 0.001000
	Average Loss: 0.000513 	Learning rate: 0.001000
	Average Loss: 0.000496 	Learning rate: 0.001000
	Average Loss: 0.000484 	Learning rate: 0.001000
	Average Loss: 0.000470 	Learning rate: 0.001000
	Average Loss: 0.000458 	Learning rate: 0.001000
	Average Loss: 0.000446 	Learning rate: 0.000500
	Average Loss: 0.000401 	Learning rate: 0.000500
	Average Loss: 0.000387 	Learning rate: 0.000500
	Average Loss: 0.000375 	Learning rate: 0.000500
	Average Loss: 0.000364 	Learning rate: 0.000500
	Average Loss: 0.000354 	Learning rate: 0.000500
	Average Loss: 0.000341 	Learning rate: 0.000500
	Average Loss: 0.000329 	Learning rate: 0.000500
	Average Loss: 0.000320 	Learning rate: 0.000500
	Average Loss: 0.000308 	Learning rate: 0.000500
	Average Loss: 0.000297 	Learning rate: 0.000250
	Average Loss: 0.000

...

	Average Loss: 0.000743 	Learning rate: 0.001000
	Average Loss: 0.000601 	Learning rate: 0.001000
	Average Loss: 0.000557 	Learning rate: 0.001000
	Average Loss: 0.000534 	Learning rate: 0.001000
	Average Loss: 0.000515 	Learning rate: 0.001000
	Average Loss: 0.000497 	Learning rate: 0.001000
	Average Loss: 0.000482 	Learning rate: 0.001000
	Average Loss: 0.000468 	Learning rate: 0.001000
	Average Loss: 0.000455 	Learning rate: 0.001000
	Average Loss: 0.000447 	Learning rate: 0.000500
	Average Loss: 0.000400 	Learning rate: 0.000500
	Average Loss: 0.000386 	Learning rate: 0.000500
	Average Loss: 0.000375 	Learning rate: 0.000500
	Average Loss: 0.000363 	Learning rate: 0.000500
	Average Loss: 0.000351 	Learning rate: 0.000500
	Average Loss: 0.000342 	Learning rate: 0.000500
	Average Loss: 0.000331 	Learning rate: 0.000500
	Average Loss: 0.000319 	Learning rate: 0.000500
	Average Loss: 0.000304 	Learning rate: 0.000500
	Average Loss: 0.000298 	Learning rate: 0.000250
	Average Loss: 0.000

  0%|          | 0/30 [00:00<?, ?it/s]

	Average Loss: 0.000746 	Learning rate: 0.001000
	Average Loss: 0.000601 	Learning rate: 0.001000
	Average Loss: 0.000561 	Learning rate: 0.001000
	Average Loss: 0.000534 	Learning rate: 0.001000
	Average Loss: 0.000515 	Learning rate: 0.001000
	Average Loss: 0.000499 	Learning rate: 0.001000
	Average Loss: 0.000483 	Learning rate: 0.001000
	Average Loss: 0.000468 	Learning rate: 0.001000
	Average Loss: 0.000458 	Learning rate: 0.001000
	Average Loss: 0.000447 	Learning rate: 0.000500
	Average Loss: 0.000402 	Learning rate: 0.000500
	Average Loss: 0.000387 	Learning rate: 0.000500
	Average Loss: 0.000376 	Learning rate: 0.000500
	Average Loss: 0.000365 	Learning rate: 0.000500
	Average Loss: 0.000354 	Learning rate: 0.000500
	Average Loss: 0.000341 	Learning rate: 0.000500
	Average Loss: 0.000331 	Learning rate: 0.000500
	Average Loss: 0.000320 	Learning rate: 0.000500
	Average Loss: 0.000311 	Learning rate: 0.000500
	Average Loss: 0.000300 	Learning rate: 0.000250
	Average Loss: 0.000

  0%|          | 0/30 [00:00<?, ?it/s]

	Average Loss: 0.000748 	Learning rate: 0.001000
	Average Loss: 0.000609 	Learning rate: 0.001000
	Average Loss: 0.000560 	Learning rate: 0.001000
	Average Loss: 0.000536 	Learning rate: 0.001000
	Average Loss: 0.000516 	Learning rate: 0.001000
	Average Loss: 0.000498 	Learning rate: 0.001000
	Average Loss: 0.000484 	Learning rate: 0.001000
	Average Loss: 0.000473 	Learning rate: 0.001000
	Average Loss: 0.000459 	Learning rate: 0.001000
	Average Loss: 0.000447 	Learning rate: 0.000500
	Average Loss: 0.000401 	Learning rate: 0.000500
	Average Loss: 0.000390 	Learning rate: 0.000500
	Average Loss: 0.000378 	Learning rate: 0.000500
	Average Loss: 0.000366 	Learning rate: 0.000500
	Average Loss: 0.000354 	Learning rate: 0.000500
	Average Loss: 0.000344 	Learning rate: 0.000500
	Average Loss: 0.000333 	Learning rate: 0.000500
	Average Loss: 0.000321 	Learning rate: 0.000500
	Average Loss: 0.000309 	Learning rate: 0.000500
	Average Loss: 0.000302 	Learning rate: 0.000250
	Average Loss: 0.000

In [ ]:
import pandas
micro = []
macro = []

for fold in range(0,5):
    csv = pandas.read_csv(f"./kfold/csv/fold-{fold}_ROC.csv")
    micro.append(csv[csv["class"] == "micro"]["ROCAUC"])
    macro.append(csv[csv["class"] == "macro"]["ROCAUC"])

In [ ]:
import numpy as np

print(f"Mean\t: {np.mean(micro)}")
print(f" SD \t: {np.std(micro) }")

Mean	: 0.9332625389099121
 SD 	: 0.0016142749576401863


In [ ]:
print(f"Mean\t: {np.mean(macro)}")
print(f" SD \t: {np.std(macro) }")

Mean	: 0.9025099895502391
 SD 	: 0.00228444830865374


In [ ]:
import pandas
micro = []
macro = []

for fold in range(0,5):
    csv = pandas.read_csv(f"./kfold/csv/fold-{fold}_PrecisionRecall.csv")
    micro.append(csv[csv["class"] == "micro"]["PrecisionRecallAUC"])
    macro.append(csv[csv["class"] == "macro"]["PrecisionRecallAUC"])

In [ ]:
import numpy as np

print(f"Mean\t: {np.mean(micro)}")
print(f" SD \t: {np.std(micro) }")

Mean	: 0.8735116600990296
 SD 	: 0.003695734927632265


In [ ]:
print(f"Mean\t: {np.mean(macro)}")
print(f" SD \t: {np.std(macro) }")

Mean	: 0.7894144812696858
 SD 	: 0.007083160749387569
